### 11/17/2023 - EC463 Prototype Test Cases

generic_name = 'lidocaine' <br>
reference_item_number = 10090846 <br>
generic_name = 'lidocaine' <br>
reference_item_number = 10104126 <br>
generic_name = 'lidocaine' <br>
reference_item_number = 10136471 <br>
<br>
<br>
generic_name = 'cefepime' <br>
reference_item_number = 10175647 <br>
generic_name = 'cefepime' <br>
reference_item_number = 10267015 <br>
generic_name = 'cefepime' <br>
reference_item_number = 10027431 <br>
<br>
<br>
generic_name = 'ondansetron' <br>
reference_item_number = 10096566 <br>
generic_name = 'ondansetron' <br>
reference_item_number = 10112532 <br>
generic_name = 'ondansetron' <br>
reference_item_number = 10126451 <br>

In [661]:
#Variable to use to filter by generic name
generic_name = 'cefepime'
#Item number of the drug to run the similarity test on
reference_item_number = 10027431

# Let's open and read the content of the uploaded file to understand its structure and data.
file_path = 'Daily Snapshot.csv'

# Reading the file
import pandas as pd

# Load the data from the CSV file
data = pd.read_csv(file_path)
data.head()  # Display the first few rows of the dataframe to understand its structure.


,Item Number – 6 digit,NDC Number,UPC Number,Constant,Customer-Specific Item Number,Description,Pack Size Divisor,Size Qty,RX/OTC Indicator,AWP Price,Acquisition Price,Retail Price,Contract Flag,Generic Description,Retail Pack Quantity,WAC Price,Item Number – 8 digit
0,1091,51672408306,351672408367,,,HYDROCORTISONE BUTYRATE 0.1% O,45000,45,R,17928,9425,0,C,hydrocortisone butyrate TOPICA,1,14342,10000009
1,1103,00023334807,300233348079,,,OZURDEX IMP 1 DS,1000,,R,164760,137300,0,N,dexamethasone INTRAOCULR IMPLA,1,137300,10083412
2,1141,00093227234,300932272347,,,AMOXICILLIN-CLAV K 400-57 MG C,20000,20,R,11751,5038,0,C,amoxicillin/potassium clav ORA,1,8814,10000013
3,1457,51991062033,351991620334,,,ANASTROZOLE 1 MG TAB 30,30000,30,R,40038,831,0,N,anastrozole ORAL TABLET 1 MG,1,900,10083430
4,1479,,070610209162,,,BERRY OINTMENT JAR 916 WHITE 1,1000,12,O,0,4641,6962,N,,1,4641,10000056


In [662]:
# Removing the specified columns and reordering the 'Item Number – 8 digit' column
columns_to_remove = ['Item Number – 6 digit', 'UPC Number', 'Constant', 
                     'Customer-Specific Item Number', 'Pack Size Divisor', 
                     'RX/OTC Indicator']

# Removing the columns
data_cleaned = data.drop(columns=columns_to_remove)

# Reordering 'Item Number – 8 digit' to the left
column_to_move = data_cleaned.pop('Item Number – 8 digit')
data_cleaned.insert(0, 'Item Number – 8 digit', column_to_move)

# Displaying the first few rows of the modified dataframe
data_cleaned.head()


,Item Number – 8 digit,NDC Number,Description,Size Qty,AWP Price,Acquisition Price,Retail Price,Contract Flag,Generic Description,Retail Pack Quantity,WAC Price
0,10000009,51672408306,HYDROCORTISONE BUTYRATE 0.1% O,45,17928,9425,0,C,hydrocortisone butyrate TOPICA,1,14342
1,10083412,00023334807,OZURDEX IMP 1 DS,,164760,137300,0,N,dexamethasone INTRAOCULR IMPLA,1,137300
2,10000013,00093227234,AMOXICILLIN-CLAV K 400-57 MG C,20,11751,5038,0,C,amoxicillin/potassium clav ORA,1,8814
3,10083430,51991062033,ANASTROZOLE 1 MG TAB 30,30,40038,831,0,N,anastrozole ORAL TABLET 1 MG,1,900
4,10000056,,BERRY OINTMENT JAR 916 WHITE 1,12,0,4641,6962,N,,1,4641


In [663]:
# Moving all price columns and the contract flag to the right
columns_to_move = ['AWP Price', 'Acquisition Price', 'Retail Price', 'WAC Price', 'Contract Flag']

# Reordering the columns
for col in columns_to_move:
    data_cleaned[col] = data_cleaned.pop(col)

# Displaying the first few rows of the rearranged dataframe
data_cleaned.head()


,Item Number – 8 digit,NDC Number,Description,Size Qty,Generic Description,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag
0,10000009,51672408306,HYDROCORTISONE BUTYRATE 0.1% O,45,hydrocortisone butyrate TOPICA,1,17928,9425,0,14342,C
1,10083412,00023334807,OZURDEX IMP 1 DS,,dexamethasone INTRAOCULR IMPLA,1,164760,137300,0,137300,N
2,10000013,00093227234,AMOXICILLIN-CLAV K 400-57 MG C,20,amoxicillin/potassium clav ORA,1,11751,5038,0,8814,C
3,10083430,51991062033,ANASTROZOLE 1 MG TAB 30,30,anastrozole ORAL TABLET 1 MG,1,40038,831,0,900,N
4,10000056,,BERRY OINTMENT JAR 916 WHITE 1,12,,1,0,4641,6962,4641,N


In [664]:
import re

# Function to split the generic description into generic name and form
def split_description(desc):
    # Find the index of the first capital letter
    match = re.search(r'[A-Z]', desc)
    if match:
        index = match.start()
        return desc[:index].strip(), desc[index:].strip()
    else:
        return desc, ''

# Applying the function to the 'Generic Description' column
data_cleaned['Generic Name'], data_cleaned['Form'] = zip(*data_cleaned['Generic Description'].apply(split_description))

# Dropping the original 'Generic Description' column
data_cleaned.drop(columns=['Generic Description'], inplace=True)

# Displaying the first few rows of the updated dataframe
data_cleaned.head()


,Item Number – 8 digit,NDC Number,Description,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form
0,10000009,51672408306,HYDROCORTISONE BUTYRATE 0.1% O,45,1,17928,9425,0,14342,C,hydrocortisone butyrate,TOPICA
1,10083412,00023334807,OZURDEX IMP 1 DS,,1,164760,137300,0,137300,N,dexamethasone,INTRAOCULR IMPLA
2,10000013,00093227234,AMOXICILLIN-CLAV K 400-57 MG C,20,1,11751,5038,0,8814,C,amoxicillin/potassium clav,ORA
3,10083430,51991062033,ANASTROZOLE 1 MG TAB 30,30,1,40038,831,0,900,N,anastrozole,ORAL TABLET 1 MG
4,10000056,,BERRY OINTMENT JAR 916 WHITE 1,12,1,0,4641,6962,4641,N,,


In [665]:
# Removing rows where the 'Generic Name' is empty or consists only of whitespace
data_cleaned = data_cleaned[data_cleaned['Generic Name'].str.strip() != '']

# Displaying the first few rows of the updated dataframe
data_cleaned.head()


,Item Number – 8 digit,NDC Number,Description,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form
0,10000009,51672408306,HYDROCORTISONE BUTYRATE 0.1% O,45,1,17928,9425,0,14342,C,hydrocortisone butyrate,TOPICA
1,10083412,00023334807,OZURDEX IMP 1 DS,,1,164760,137300,0,137300,N,dexamethasone,INTRAOCULR IMPLA
2,10000013,00093227234,AMOXICILLIN-CLAV K 400-57 MG C,20,1,11751,5038,0,8814,C,amoxicillin/potassium clav,ORA
3,10083430,51991062033,ANASTROZOLE 1 MG TAB 30,30,1,40038,831,0,900,N,anastrozole,ORAL TABLET 1 MG
5,10083433,67457022005,ISOSULFAN BLUE 1% SDV 6X5 ML,6X5,1,763841,587712,0,636534,N,isosulfan blue,SUBCUT VIAL 1 %


In [666]:
# Function to split the description into name and size
def split_description_on_number(desc):
    # Find the index of the first number
    match = re.search(r'\d', desc)
    if match:
        index = match.start()
        return desc[:index].strip(), desc[index:].strip()
    else:
        return desc, ''

# Applying the function to the 'Description' column
data_cleaned['Name'], data_cleaned['Size'] = zip(*data_cleaned['Description'].apply(split_description_on_number))

# Dropping the original 'Description' column
data_cleaned.drop(columns=['Description'], inplace=True)

# Displaying the first few rows of the updated dataframe
data_cleaned.head()


,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form,Name,Size
0,10000009,51672408306,45,1,17928,9425,0,14342,C,hydrocortisone butyrate,TOPICA,HYDROCORTISONE BUTYRATE,0.1% O
1,10083412,00023334807,,1,164760,137300,0,137300,N,dexamethasone,INTRAOCULR IMPLA,OZURDEX IMP,1 DS
2,10000013,00093227234,20,1,11751,5038,0,8814,C,amoxicillin/potassium clav,ORA,AMOXICILLIN-CLAV K,400-57 MG C
3,10083430,51991062033,30,1,40038,831,0,900,N,anastrozole,ORAL TABLET 1 MG,ANASTROZOLE,1 MG TAB 30
5,10083433,67457022005,6X5,1,763841,587712,0,636534,N,isosulfan blue,SUBCUT VIAL 1 %,ISOSULFAN BLUE,1% SDV 6X5 ML


In [667]:
# Filtering the dataset for items with the exact generic name "lidocaine"
data_lidocaine = data_cleaned[data_cleaned['Generic Name'].str.lower() == generic_name]

data_lidocaine


,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form,Name,Size
388,10087370,63323034020,10,1,13008,10009,0,10840,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10
1755,10009785,00264319511,24X50,1,85968,61362,0,71640,C,cefepime,HCl in dextrose 5 % I,CEFEPIME-DEXT,2 GM DUP BAG 24X
1756,10009786,00264319311,24X50,1,56678,40458,0,47232,C,cefepime,HCl in dextrose 5 % I,CEFEPIME-DEXT,1 GM DUP BAG 24X
6346,10127872,44567024110,10,1,11760,4414,0,9800,C,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10
6347,10127874,44567024010,10,1,6840,2368,0,5700,C,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10
10594,10027431,63323032620,10,1,7620,5863,0,6350,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10
16680,10175647,60505614600,,1,2033,467,0,505,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1GM SDV
16681,10175648,60505614700,,1,4036,1305,0,1413,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2GM SDV
16682,10175680,60505614604,10,1,20334,4659,0,5045,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10
17903,10177987,25021012120,10,1,7200,4033,0,6000,C,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10


In [668]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Selecting the reference item

reference_item = data_lidocaine[data_lidocaine['Item Number – 8 digit'] == reference_item_number]

# Check if the reference item is found
if reference_item.empty:
    similarity_results = "Reference item not found in the dataset."
else:
    # Extracting the form of the reference item
    reference_form = reference_item.iloc[0]['Form']

    # Preparing data for TF-IDF vectorization (including the reference form)
    forms = data_lidocaine['Form'].tolist()
    forms.insert(0, reference_form)

    # Vectorizing the forms using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(forms)

    # Calculating cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Adding the similarity scores to the lidocaine dataset
    # We exclude the first element of cosine_similarities as it is the similarity of the reference item with itself
    data_lidocaine['Similarity'] = cosine_similarities

    # Displaying the dataset with similarity scores
    similarity_results = data_lidocaine

  # Display the first few rows of the results

# Filtering the dataset to show only items with a similarity score above 0.9
similarity_items = data_lidocaine[data_lidocaine['Similarity'] > 0.9]

similarity_items



/var/folders/1q/j6g9lj1x5gz1fvb0sstc6rnm0000gn/T/ipykernel_87787/3440652549.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lidocaine['Similarity'] = cosine_similarities


,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form,Name,Size,Similarity
388,10087370,63323034020,10,1,13008,10009,0,10840,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0
6346,10127872,44567024110,10,1,11760,4414,0,9800,C,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0
6347,10127874,44567024010,10,1,6840,2368,0,5700,C,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0
10594,10027431,63323032620,10,1,7620,5863,0,6350,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0
16680,10175647,60505614600,,1,2033,467,0,505,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1GM SDV,1.0
16681,10175648,60505614700,,1,4036,1305,0,1413,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2GM SDV,1.0
16682,10175680,60505614604,10,1,20334,4659,0,5045,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0
17903,10177987,25021012120,10,1,7200,4033,0,6000,C,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0
19138,10180436,60505614704,10,1,40356,13047,0,14130,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0
19601,10237777,00409973510,10,1,12060,5725,0,10050,C,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0


In [669]:
# Extracting the size of the reference item
reference_size = reference_item.iloc[0]['Size']

# Preparing data for TF-IDF vectorization (including the reference size)
sizes = similarity_items['Size'].tolist()
sizes.insert(0, reference_size)

# Vectorizing the sizes using TF-IDF
tfidf_matrix_sizes = vectorizer.fit_transform(sizes)

# Calculating cosine similarity for sizes
cosine_similarities_sizes = cosine_similarity(tfidf_matrix_sizes[0:1], tfidf_matrix_sizes[1:]).flatten()

# Adding the similarity scores for sizes to the high_similarity_items dataset
# We exclude the first element of cosine_similarities_sizes as it is the similarity of the reference item with itself
similarity_items['Size Similarity'] = cosine_similarities_sizes

# Displaying the dataset with size similarity scores
high_similarity_items_filtered_by_size = similarity_items[similarity_items['Size Similarity'] > 0]
high_similarity_items_filtered_by_size  # Display the first few rows of the results


/var/folders/1q/j6g9lj1x5gz1fvb0sstc6rnm0000gn/T/ipykernel_87787/672923166.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similarity_items['Size Similarity'] = cosine_similarities_sizes


,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form,Name,Size,Similarity,Size Similarity
388,10087370,63323034020,10,1,13008,10009,0,10840,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0,1.000000
6346,10127872,44567024110,10,1,11760,4414,0,9800,C,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0,1.000000
6347,10127874,44567024010,10,1,6840,2368,0,5700,C,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0,1.000000
10594,10027431,63323032620,10,1,7620,5863,0,6350,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0,1.000000
16680,10175647,60505614600,,1,2033,467,0,505,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1GM SDV,1.0,0.163470
16681,10175648,60505614700,,1,4036,1305,0,1413,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2GM SDV,1.0,0.163470
16682,10175680,60505614604,10,1,20334,4659,0,5045,N,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0,1.000000
17903,10177987,25021012120,10,1,7200,4033,0,6000,C,cefepime,HCl INJECTION VIAL 1,CEFEPIME,1 GM SDV 10,1.0,1.000000
19138,10180436,60505614704,10,1,40356,13047,0,14130,N,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0,1.000000
19601,10237777,00409973510,10,1,12060,5725,0,10050,C,cefepime,HCl INJECTION VIAL 2,CEFEPIME,2 GM SDV 10,1.0,1.000000
